### 지니차트 Top200

In [ ]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

1. 데이터 가져오기

In [ ]:
url = 'https://www.genie.co.kr/chart/top200'

In [ ]:
# Chrome User-Agent
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
result = requests.get(url, headers=header)
soup = BeautifulSoup(result.text, 'html.parser')
soup

##### 2. 원하는 데이터 찾기

In [ ]:
trs = soup.select('tr.list')
trs

- 가져온 정보에서 rank, img url, title, artist, album 찾기

In [ ]:
tr = trs[0]

In [ ]:
# rank
rank = tr.select_one('.number').get_text().split('\n')[0].strip()       # strip : \n포함 문자열 양옆 공백 제거
rank

In [ ]:
# img url
img = tr.select_one('.cover > img')['src']
img = f'https:{img}'
img

In [ ]:
# title
title = tr.select_one('.title.ellipsis').get_text().strip()
title

# 19금 노래 제목 찾는법 : title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()

In [ ]:
# 위의 두 코드를 합치면
# 방법1)
title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()

# 방법2)
try:
    title = tr.select_one('.title.ellipsis').get_text().strip()
except:
    title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()

In [ ]:
# artists
artists = tr.select_one('.artist.ellipsis').get_text()
artists

In [ ]:
# album
album = tr.select_one('.albumtitle.ellipsis').get_text()
album

In [ ]:
# 방법1. 반복문으로 딕셔너리의 리스트로 가져오기
line = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0].strip()       # strip : \n포함 문자열 양옆 공백 제거
    img = tr.select_one('.cover > img')['src']
    img = f'https:{img}'
    title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
    artists = tr.select_one('.artist.ellipsis').get_text()
    album = tr.select_one('.albumtitle.ellipsis').get_text()
    line.append({'rank':rank, 'img':img, 'title':title, 'artists':artists, 'album':album})
df_line = pd.DataFrame(line)
df_line.set_index('rank', inplace=True)
df_line

In [ ]:
# 방법2. 개별 리스트로 가져오기
rank_list, img_list, title_list, artists_list, album_list = [],[],[],[],[]
for tr in trs:
    rank_list.append(tr.select_one('.number').get_text().split('\n')[0].strip())       # strip : \n포함 문자열 양옆 공백 제거
    img_list.append(f"https:{tr.select_one('.cover > img')['src']}")
    title_list.append(tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip())
    artists_list.append(tr.select_one('.artist.ellipsis').get_text())
    album_list.append(tr.select_one('.albumtitle.ellipsis').get_text())

# - > DF에 개별리스트 추가

#### 6. 모든페이지에 대해 반복하기

In [ ]:
from datetime import datetime
now = datetime.now()
now

In [ ]:
ymd = now.strftime('%Y%m%d')
hh = now.strftime('%H')
ymd, hh

In [ ]:
from tqdm import tqdm

In [ ]:
line1 = []
for i in tqdm(range(1, 5)):
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd={ymd}&hh={hh}&rtm=Y&pg={i}'
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
    result = requests.get(url, headers=header)
    soup = BeautifulSoup(result.text, 'html.parser')
    trs = soup.select('tr.list')
    for tr in trs:
        rank = tr.select_one('.number').get_text().split('\n')[0].strip()       # strip : \n포함 문자열 양옆 공백 제거
        img = tr.select_one('.cover > img')['src']
        img = f'https:{img}'
        title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
        artists = tr.select_one('.artist.ellipsis').get_text()
        album = tr.select_one('.albumtitle.ellipsis').get_text()
        line1.append({'rank':rank, 'img':img, 'title':title, 'artists':artists, 'album':album})

df1 = pd.DataFrame(line1)

In [ ]:
df1.head(60).tail(10)

In [ ]:
df1.to_csv(f'data/{ymd}{hh}_지니차트.csv', index=False)